In [1]:
import sys,os

#sys.path.append('/home/arash/VRdataCleaning/DeepSurv/')

import importlib
import deepsurv
from statsmodels.stats.outliers_influence import variance_inflation_factor    
import argparse
import uuid
import pickle
import json

import numpy as np
import lasagne
import optunity

import logging
from logging import handlers


importlib.reload(deepsurv)

from deepsurv import deep_surv, utils, viz

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger



import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
data = pd.read_pickle('ALLDATA.pkl')
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'Vrexp_Yes']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later


data=data.apply(pd.to_numeric, errors='coerce')

def FullAV(row):
    if row['AV']==100:
        return 1
    else:
        return 0
def MixedAV(row):
    if row['AV']>0 and row['AV']<100:
        return 1
    else:
        return 0   
def HDV(row):
    if row['AV']==0:
        return 1
    else:
        return 0   
data=data[data.Occupation_kid==0]                         #remove kids


data['FullAV']=data.apply (lambda row: FullAV(row), axis=1)
data['Mixed']=data.apply (lambda row: MixedAV(row), axis=1)
#data['HDV']=data.apply (lambda row: HDV(row), axis=1)
data['over50']=data['Age_50 - 59']+data['Age_60+']
data=data.drop(columns=['Age_9-12', 'Age_15-18', 'Age_12-15','Occupation_kid','AV','Age_50 - 59','Age_60+'])
data['E']=1


In [4]:
def dataframe_to_deepsurv_ds(df, event_col = 'E', time_col = 'Wait Time (s)'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }


data = dataframe_to_deepsurv_ds(data, event_col = 'E', time_col= 'Wait Time (s)')

In [5]:
def load_logger(logdir):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    # Print to Stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # Print to Log file
    fh = logging.FileHandler(os.path.join(logdir, 'log_' + str(uuid.uuid4())))
    fh.setFormatter(format)
    logger.addHandler(fh)

    return logger

def load_box_constraints(file):
    with open(file, 'rb') as fp:
        return json.loads(fp.read())

def save_call_log(file, call_log):
    with open(file, 'wb') as fp:
        pickle.dump(call_log, fp)

In [6]:
def get_objective_function(num_epochs, logdir, update_fn = lasagne.updates.sgd):
    '''
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to 
    use as hyper-parameters.

    The objective function runs a DeepSurv model on the training data and evaluates it against the
    test set for validation. The result of the function call is the validation concordance index 
    (which Optunity tries to optimize)
    '''
    def format_to_deepsurv(x, y):
        return {
            'x': x,
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }

    def get_hyperparams(params):
        hyperparams = {
            'batch_norm': False,
            'activation': 'rectify',
            'standardize': True
        }
        # @TODO add default parameters and only take necessary args from params
        # protect from params including some other key

        if 'num_layers' in params and 'num_nodes' in params:
            params['hidden_layers_sizes'] = [int(params['num_nodes'])] * int(params['num_layers'])
            del params['num_layers']
            del params['num_nodes']

        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
            
        if 'n_in' in params:
            params['n_in'] = int(params['n_in'])
            

        hyperparams.update(params)
        return hyperparams

    def train_deepsurv(x_train,y_train,x_test,y_test,
        **kwargs):
        hyperparams = get_hyperparams(kwargs)
        n_in=int(hyperparams['n_in'])

        #select number of features
        #ReliefAvg = pd.read_pickle('/home/arash/VRdataCleaning/ReliefAvg.pkl')
        ReliefAvg = pd.read_pickle('/home/arash/VRdataCleaning/Surf.pkl')
        x_train=x_train[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]
        x_test=x_test[:,ReliefAvg.sort_values(['Importance'],ascending=False).index[0:n_in]]
        



        
        
        # Standardize the datasets
        train_mean = x_train.mean(axis = 0)
        train_std = x_train.std(axis = 0)

        x_train = (x_train - train_mean) / train_std
        x_test = (x_test - train_mean) / train_std

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)

        

        # Set up Tensorboard loggers
        # TODO improve the model_id for Tensorboard to better partition runs
        model_id = str(hash(str(hyperparams)))
        run_id = model_id + '_' + str(uuid.uuid4())
        logger = TensorboardLogger('hyperparam_search', 
            os.path.join(logdir,"tensor_logs", model_id, run_id))

        network = deep_surv.DeepSurv(**hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs, logger=logger, 
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        main_logger.info('Run id: %s | %s | C-Index: %f | Train Loss %f' % (run_id, str(hyperparams), result, metrics['loss'][-1][1]))
        return result

    return train_deepsurv

In [7]:
NUM_EPOCHS = 10
NUM_FOLDS = 2
logdir='/home/arash/VRdataCleaning/logs'
x = data['x']
e = data['e']
t = data['t']
y = np.column_stack((e, t))


In [8]:
#    global main_logger
main_logger = load_logger(logdir)


In [9]:
#    main_logger.debug('Loading dataset: ' + args.dataset)
box_constraints = load_box_constraints('/home/arash/VRdataCleaning/box_constraints.0.json')


In [10]:
opt_fxn = get_objective_function(NUM_EPOCHS, logdir, 
                                 utils.get_optimizer_from_str('adam'))

In [11]:
opt_fxn = optunity.cross_validated(x=x, y=y, num_folds=NUM_FOLDS)(opt_fxn)

In [12]:
#    main_logger.debug('Maximizing C-Index. Num_iterations: %d' % args.num_evals)
opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=10,
        solver_name='sobol',
        **box_constraints)

#    main_logger.debug('Optimal Parameters: ' + str(opt_params))
#    main_logger.debug('Saving Call log...')

2019-05-17 16:04:31,679 - __main__ - INFO - Run id: -8551354460127966334_16ab0d81-31a9-4dd0-8864-b9e5ae800214 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.000549306640625, 'learning_rate': 1.163435168279781e-05, 'n_in': 17, 'momentum': 0.84962158203125, 'L2_reg': 2.29050341796875, 'dropout': 0.051080078125000004, 'hidden_layers_sizes': [62, 62, 62]} | C-Index: 0.482167 | Train Loss 9.546736
2019-05-17 16:04:34,018 - __main__ - INFO - Run id: -8551354460127966334_b3164aae-5350-4629-8fb7-a0ab573b4b22 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.000549306640625, 'learning_rate': 1.163435168279781e-05, 'n_in': 17, 'momentum': 0.84962158203125, 'L2_reg': 2.29050341796875, 'dropout': 0.051080078125000004, 'hidden_layers_sizes': [62, 62, 62]} | C-Index: 0.473292 | Train Loss 10.159004
2019-05-17 16:04:37,013 - __main__ - INFO - Run id: -2539921375529510163_8a8b17c7-3732-492a-852f-d49ec5efd026 | {'batch_norm': False,

2019-05-17 16:05:21,994 - __main__ - INFO - Run id: 2258294675142179016_a4385b8f-8f56-4612-acdc-f23b709d41a5 | {'batch_norm': False, 'activation': 'rectify', 'standardize': True, 'lr_decay': 0.000982431640625, 'learning_rate': 2.1539825840327113e-07, 'n_in': 5, 'momentum': 0.89602783203125, 'L2_reg': 0.7590971679687499, 'dropout': 0.088205078125, 'hidden_layers_sizes': [65, 65, 65]} | C-Index: 0.507658 | Train Loss 7.451966


In [ ]:
save_call_log(os.path.join(logdir, 'optunity_log_%s.pkl' % (str(uuid.uuid4()))), call_log._asdict())

In [15]:
opt_params


{'lr_decay': 0.000920556640625,
 'learning_rate': -3.4492578125,
 'num_nodes': 93.70751953125,
 'num_layers': 3.232529296875,
 'n_in': 7.931884765625,
 'momentum': 0.9053090820312499,
 'L2_reg': 1.67794091796875,
 'dropout': 0.174830078125}

In [14]:
call_log

NT(optimum=0.5509384824606196, stats=OrderedDict([('num_evals', 10), ('time', 75.40993765904568)]), call_log={'args': {'num_nodes': [62.77001953125, 50.39501953125, 75.14501953125, 68.95751953125, 93.70751953125, 81.33251953125, 56.58251953125, 78.23876953125, 53.48876953125, 65.86376953125], 'n_in': [17.213134765625, 13.500634765625001, 6.075634765625001, 15.356884765625, 7.931884765625, 11.644384765625002, 19.069384765625003, 16.285009765625002, 8.860009765625001, 5.147509765625], 'learning_rate': [-4.9342578125, -5.9242578125000005, -3.9442578125, -5.4292578125, -3.4492578125, -6.419257812500001, -4.4392578125, -5.6767578125, -3.6967578125, -6.6667578125], 'lr_decay': [0.000549306640625, 0.000796806640625, 0.000301806640625, 0.000425556640625, 0.000920556640625, 0.000673056640625, 0.00017805664062500002, 0.00023993164062500002, 0.000734931640625, 0.000982431640625], 'momentum': [0.84962158203125, 0.88674658203125, 0.8124965820312501, 0.83105908203125, 0.9053090820312499, 0.868184082

In [13]:
Relief=pd.read_pickle('/home/arash/VRdataCleaning/Surf.pkl')
Relief.sort_values(['Importance'],ascending=False)

,Covariate,Importance
28,shopwalk_No,0.029762
19,Education_High school diploma,0.0249818
33,over50,0.0219528
23,mode_Car,0.0192792
27,workwalk_Yes,0.0186831
3,Mean Arrival Rate,0.0180707
20,Education_Masters degree,0.0172059
6,Snowy,0.0111135
11,Age_18 - 24,0.0109934
16,Occupation_Unemployed,0.0102918


In [ ]:
#Model with specific hyper parameters  #deepsurv needs to be fixed before use
hyperparams = {
    'L2_reg': 3.0,
    'batch_norm': False,
    'dropout': 0.02,
    'hidden_layers_sizes': [90,90,90],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': 10,
    'standardize': True
}

# Create an instance of DeepSurv using the hyperparams defined above
model = deep_surv.DeepSurv(**hyperparams)



experiment_name = 'Wait Time analysis'
logdir = './logs/tensorboard/'
logger = TensorboardLogger(experiment_name, logdir=logdir)

# Now we train the model
update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. \
                                            # Check out http://lasagne.readthedocs.io/en/latest/modules/updates.html \
                                            # for other optimizers to use
n_epochs = 2000

# If you have validation data, you can add it as the second parameter to the function
metrics = model.train(data, n_epochs=n_epochs, logger=logger, update_fn=update_fn)

# Print the final metrics
print('Train C-Index:', metrics['c-index'][-1])
# print('Valid C-Index: ',metrics['valid_c-index'][-1])

# Plot the training / validation curves
viz.plot_log(metrics)       